In [ ]:
pip install pyspark koalas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.2/593.2 kB 11.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=9719a93346e9c814eeaf31e3aae8c5fc1d16ab11d8ada2ba7a88f0e056830a1f
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 44.8 MB/s eta 0:00:00


In [ ]:
pip install textblob

In [ ]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col, split
from pyspark.sql.types import StringType, ArrayType
from pyspark.sql.functions import udf
from textblob import TextBlob

# Spark 세션 생성
spark = SparkSession.builder.appName("SentimentAnalysis").getOrCreate()

# CSV 파일 불러오기
file_path = "/content/inti_op.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

# 'text' 컬럼을 선택하고 배열로 변환
df = df.withColumn("text", split(df["text"], ",\s*").cast(ArrayType(StringType())))

# 'text' 컬럼을 선택하고 explode를 사용하여 배열 풀기
df_text = df.select(explode(col("text")).alias("text"))

# TextBlob을 사용한 감정 분석 함수 정의
def analyze_sentiment(text):
    analysis = TextBlob(text)
    # 감정 점수에 따라 긍정/중립/부정으로 분류
    if analysis.sentiment.polarity > 0.5:
        return "positive"
    elif analysis.sentiment.polarity == 0.5:
        return "neutral"
    else:
        return "negative"

# PySpark DataFrame의 특정 컬럼에 사용자 정의 함수를 적용하기 위해 udf 생성
sentiment_udf = udf(analyze_sentiment, StringType())

# 'text' 컬럼에 대해 감정 분석 수행하여 'sentiment' 컬럼 추가
df_text = df_text.withColumn("sentiment", sentiment_udf("text"))

# 결과 출력
df_text.show(truncate=False)

# 결과를 CSV 파일로 저장
result_path = "/content/inti_result.csv"
df_text.select("text", "sentiment").coalesce(1).write.mode("overwrite").option("header", "true").csv(result_path)


+-----------------------------------------------------------------------------------------------------+---------+
|text                                                                                                 |sentiment|
+-----------------------------------------------------------------------------------------------------+---------+
|나는 만약 김포에서 서울 산다고 하면 좀 어이없을 듯                                                   |negative |
|김포 서울 편입한다더니 이젠 양산김해 부산 편입까지 말나오는게 진짜 어이없다                          |negative |
|내년에 지방직 공무원 안뽑나                                                                          |negative |
|김포뿐 아니라 과천                                                                                   |negative |
|하남                                                                                                 |negative |
|고양 등도 서울편입 논의한다던데… 이러면 경기도 다 죽는 거 아닌가… 알짜배기 지자체 다 떼서 서울 주면… |negative |
|그냥 경기 분도                                                                                  

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col, split
from pyspark.sql.types import StringType, ArrayType
from pyspark.sql.functions import udf
from textblob import TextBlob

# Spark 세션 생성
spark = SparkSession.builder.appName("SentimentAnalysis").getOrCreate()

# CSV 파일 불러오기
file_path = "/content/seoulnp_new.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

# 'text' 컬럼을 선택하고 배열로 변환
df = df.withColumn("text", split(df["text"], ",\s*").cast(ArrayType(StringType())))

# 'text' 컬럼을 선택하고 explode를 사용하여 배열 풀기
df_text = df.select(explode(col("text")).alias("text"))

# TextBlob을 사용한 감정 분석 함수 정의
def analyze_sentiment(text):
    analysis = TextBlob(text)
    # 감정 점수에 따라 긍정/중립/부정으로 분류
    if analysis.sentiment.polarity > -0.1:
        return "positive"
    elif analysis.sentiment.polarity == -0.1:
        return "neutral"
    else:
        return "negative"

# PySpark DataFrame의 특정 컬럼에 사용자 정의 함수를 적용하기 위해 udf 생성
sentiment_udf = udf(analyze_sentiment, StringType())

# 'text' 컬럼에 대해 감정 분석 수행하여 'sentiment' 컬럼 추가
df_text = df_text.withColumn("sentiment", sentiment_udf("text"))

# 결과 출력
df_text.show(truncate=False)

# 결과를 CSV 파일로 저장
result_path = "/content/seoulnp_sentimen1.csv"
df_text.select("text", "sentiment").coalesce(1).write.mode("overwrite").option("header", "true").csv(result_path)


+-----------------------------------------------------------------------+---------+
|text                                                                   |sentiment|
+-----------------------------------------------------------------------+---------+
|국토장관 후보자                                                        |positive |
|‘메가서울’ 논란에 “국토 효율화 위해 최선”                              |positive |
|‘김포 서울 편입’논란에 힘 빠진 경기북부특별자치도 재점화               |positive |
|김포                                                                   |positive |
|서울 편입 잰걸음… 기후동행카드도 동참                                  |positive |
|김포시                                                                 |positive |
|서울 편입 행정절차 본격화…“주민투표 건의”                              |positive |
|[서울광장] 메가서울과 지방균형발전/오일만 세종취재본부장               |positive |
|김기현 “野 막가파식 특검·습관성 묻지마 탄핵으로 국회 마비”             |positive |
|‘아파트 사잇길’ 심층 분석 인상적… 경제 이슈 종합적으로 다뤄야          |positive |
|‘동부경남 소외론’에 박완수 지사 “과거 도정 관심 부족...균형발전 도모”  |positive |
|[열린세상] 서울

In [ ]:
'''
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col, split
from pyspark.sql.types import StringType, ArrayType

# Spark 세션 생성
spark = SparkSession.builder.appName("SentimentAnalysis").getOrCreate()

# CSV 파일 불러오기

file_path = "/content/seoulnp_new.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

# '0' 컬럼을 선택하고 배열로 변환
df = df.withColumn("0", split(df["0"], ",\s*").cast(ArrayType(StringType())))

# '0' 컬럼을 선택하고 explode를 사용하여 배열 풀기
df_text = df.select(explode(col("0")).alias("text"))

# TextBlob을 사용한 감정 분석 함수 정의
def analyze_sentiment(text):
    analysis = TextBlob(text)
    # 감정 점수가 0.3보다 크면 긍정, 작으면 부정으로 분류
    return "positive" if analysis.sentiment.polarity > -0.3 else "negative"

# PySpark DataFrame의 특정 컬럼에 사용자 정의 함수를 적용하기 위해 udf 생성
sentiment_udf = udf(analyze_sentiment, StringType())

# 'text' 컬럼에 대해 감정 분석 수행하여 'sentiment' 컬럼 추가
df_text = df_text.withColumn("sentiment", sentiment_udf("text"))

# 결과 출력
df_text.show(truncate=False)

# 결과를 CSV 파일로 저장
#result_path = "gimponews_sentiment.csv"
result_path = "seoulnp_sentiment1.csv"
df_text.select("text", "sentiment").coalesce(1).write.mode("overwrite").option("header", "true").csv(result_path)
'''

AnalysisException: ignored

In [ ]:
#새로운 CSV 파일 생성
new_file_content = '''text
"국토장관 후보자, ‘메가서울’ 논란에 “국토 효율화 위해 최선”"
"‘김포 서울 편입’논란에 힘 빠진 경기북부특별자치도 재점화"
"김포, 서울 편입 잰걸음… 기후동행카드도 동참"
"김포시, 서울 편입 행정절차 본격화…“주민투표 건의”"
"[서울광장] 메가서울과 지방균형발전/오일만 세종취재본부장"
"김기현 “野 막가파식 특검·습관성 묻지마 탄핵으로 국회 마비”"
"‘아파트 사잇길’ 심층 분석 인상적… 경제 이슈 종합적으로 다뤄야"
"‘동부경남 소외론’에 박완수 지사 “과거 도정 관심 부족...균형발전 도모”"
"[열린세상] 서울 편입보다 시도 통합이 먼저다/하혜수 경북대 행정학부 교수"
"與 “메가시티 비수도권 차별 아냐…‘제로섬’ 아닌 ‘윈윈 게임’”"
"‘메가시티’ 보폭 넓히는 오세훈, ‘북자도’ 홍보 집중 김동연…엇갈린 행보 주목"
"‘행정통합 논의 다음에’...국힘 뉴시티 특위위원장 경남 방문 취소"
"[열린세상] ‘문화의 분권화’ 시대로 가자/이종수 연세대 행정대학원장"
"野 ‘김포 5호선 연장 예타 면제법안’ 소위 단독 의결"
"김영미 교수, 25일 영광에서 출판기념회"
"국민의힘 ‘1호 영입’ 공들인 ‘113만 유튜버’ 유현준 “거절”"
"김동연 “북부특별자치도 주민투표 12월중순까지 정부서 답달라”"
"고양시장 만난 오세훈 “수도권 재편 논의할 정부 협의체를”"
"與 뉴시티특위, 서울 일괄 편입 ‘행정통합특별법’ 추진"
"서준오 서울시의원 “시대와 시민에 역행…오세훈 시장 ‘서울공화국’ 중단해야”"
"부산 상공인·시민단체, 산업은행 부산 이전 “정기국회 내 처리” 촉구"
"왕정순 서울시의원 “김포시 서울 편입 논란에 서울연구원 정치적 활용 우려”"
"“구리시, 과천시도…서울 편입될까요?” 민원 ‘봇물’"
"김포시 “시민 68%가 서울 편입 찬성” 여론조사 결과발표"
"“김포, 2025년부터 서울 편입… 농어촌 특례는 2030년까지”"
"‘김포 편입’ 견해차만 확인 후 헤어진 서울·경기·인천 단체장"
"경기도와 서울시 간 관할구역 변경에 관한 특별법률안 접수 박완수 경남지사 “결혼 우격다짐으로 안 돼...행정통합도 마찬가지”"
"與, 서울·김포 통합 특별법 오늘 발의…농어촌특례 폐지 유예"
"與 “서울·김포 통합 특별법 발의”…농어촌특례 유지"
"[서울 on] 민의로 포장한 정치/명희진 정치부 기자"
" "
'''

# 파일 경로 지정
new_file_path = "seoulnp_new.csv"

# 파일 저장
with open(new_file_path, "w") as new_file:
    new_file.write(new_file_content)

In [ ]:
'''
# 새로운 CSV 파일 생성
new_file_content = '''text
"[서울편입, 무엇이 좋아지나-보건]서울 편입되면 ‘아이 낳고 기르기 좋은 도시’ 된다"
"김병수 김포시장 “서울 편입, 김포의 정체성은 우리가 만들어 갈 것”"
"김병수 김포시장 “서울편입, 서울과 김포 윈-윈할 수 있도록 최선”"
"[서울편입, 무엇이 좋아지나-복지]서울편입되면 ＇보다 살기 좋은 김포＇ 된다"
"김병수 김포시장 “서울편입, 김포시민들의 삶의 질 달라질 것”"
"김포시, 시민 소통 박차.. 서울편입 시민 관심 고조"
"[서울편입, 무엇이 좋아지나 – 교육]서울 편입되면 ‘고교평준화 적용, 특목고·자사고 등 선택 폭 확대’"
"[서울편입, 무엇이 좋아지나-문화예술]서울편입되면 ＇일상이 문화가 되는 김포＇ 된다"
"‘소통, 또 소통’ 김병수 김포시장에 단체장들 “서울편입 긍정적”"
"김병수 김포시장 “서울편입, 총선용 아냐.. 김포특수상황 이해 필요”"
"김병수 김포시장 소통 박차, 시민들 “서울편입이 타당”"
"김포시민 68% “김포서울편입 찬성”"
"김포시, 서울시 편입 편파·오보 종편방송사 언론중재위 제소"
"[서울편입, 무엇이 좋아지나-보육]서울편입되면 ＇아이 키우기 좋은 김포＇ 된다"
"김병수 김포시장 “서울편입은 김포시민 삶의 질 향상이 기준”,서울편입·5호선 김포연장 밀도있는 소통 오가"
"김포 ‘서울편입’, 세수 큰 변동 없다"
"김포시, 통진읍 마송공영주차장 편입 사유지 소유권이전등기 국가소송 승소"
"김포시, 포내공원 구 국도 편입 토지 3,137㎡ 소유권 환수"
"김포시, 대벽저류지 편입 토지 2,921㎡ 소유권 환수"
'''

# 파일 경로 지정
new_file_path = "gimponews_new.csv"

# 파일 저장
with open(new_file_path, "w") as new_file:
    new_file.write(new_file_content)
'''

In [ ]:
#새로운 파일을 사용하여 DataFrame 생성
df_new = spark.read.option("header", "true").csv(new_file_path)

# 'text' 열에 대해 감정 분석 수행하여 'sentiment' 열 추가
df_new = df_new.withColumn("sentiment", sentiment_udf("text"))

# 결과 출력
df_new.show(truncate=False)

# 결과를 CSV 파일로 저장 (덮어쓰기 모드로 설정)
result_path_new = "/content/seoulnp_sentimen1.csv"
df_new.select("text", "sentiment").coalesce(1).write.mode("overwrite").option("header", "true").csv(result_path_new)

+-------------------------------------------------------------------------+---------+
|text                                                                     |sentiment|
+-------------------------------------------------------------------------+---------+
|국토장관 후보자, ‘메가서울’ 논란에 “국토 효율화 위해 최선”               |positive |
|‘김포 서울 편입’논란에 힘 빠진 경기북부특별자치도 재점화                 |positive |
|김포, 서울 편입 잰걸음… 기후동행카드도 동참                              |positive |
|김포시, 서울 편입 행정절차 본격화…“주민투표 건의”                        |positive |
|[서울광장] 메가서울과 지방균형발전/오일만 세종취재본부장                 |positive |
|김기현 “野 막가파식 특검·습관성 묻지마 탄핵으로 국회 마비”               |positive |
|‘아파트 사잇길’ 심층 분석 인상적… 경제 이슈 종합적으로 다뤄야            |positive |
|‘동부경남 소외론’에 박완수 지사 “과거 도정 관심 부족...균형발전 도모”    |positive |
|[열린세상] 서울 편입보다 시도 통합이 먼저다/하혜수 경북대 행정학부 교수  |positive |
|與 “메가시티 비수도권 차별 아냐…‘제로섬’ 아닌 ‘윈윈 게임’”               |positive |
|‘메가시티’ 보폭 넓히는 오세훈, ‘북자도’ 홍보 집중 김동연…엇갈린 행보 주목|positive |
|‘행정통합 논의 다음에’...국힘 뉴시티 특위위원장 경남 방문 취소           |positive |
|[

In [1]:
# 'sentiment' 컬럼만 선택
df_sentiment = df_new.select("sentiment")

# 결과를 CSV 파일로 저장 (덮어쓰기 모드로 설정)
result_path_sentiment = "gimponews_sentiment1_new.csv"
df_sentiment.coalesce(1).write.mode("overwrite").option("header", "true").csv(result_path_sentiment)

# CSV 파일 읽기
df_result_sentiment = spark.read.csv(result_path_sentiment, header=True)

# 데이터프레임 내용 확인
df_result_sentiment.show()


NameError: ignored